In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# bertscore = load("bertscore") 
# tokenizer =  AutoTokenizer.from_pretrained('facebook/bart-base',cahe_dir='/ssd_scratch/cvit/aparna/bart')
# model = AutoModelForSeq2SeqLM.from_pretrained("/ssd_scratch/cvit/aparna/mbart_simplification_final")

/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import evaluate
import numpy as np
from tqdm import tqdm
import json
from nltk.tokenize import RegexpTokenizer
bleu_metric = evaluate.load("sacrebleu")
sari_metric = evaluate.load("sari")

def tokenize_sentence(arg):
    encoded_arg =tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    print(len(eval_arg[0]),len(eval_arg[1]),len(eval_arg[2]))
    print(len(eval_arg))
    text_inputs = eval_arg[0]
    text_preds = eval_arg[1]
    text_labels = eval_arg[2]
    texts_bleu =[text.strip() for text in text_preds]
    labels_bleu = [[text.strip()] for text in text_labels[0]]
    result = bleu_metric.compute(predictions=texts_bleu, references=text_inputs)
    return result["score"],sari_metric.compute(
        predictions=text_preds,
        references=text_labels,
        sources=text_inputs,
    )['sari']

In [3]:
import pandas as pd
test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()


metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1

for  i  in tqdm(range(len(texts))):
    text = texts[i]
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=num_b,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
   # print("====================================")
    inpu.extend([text]*10)
    c =[]
    for x, beam in enumerate(beam_outputs):
        #print("{}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
        c.append(tokenizer.decode(beam, skip_special_tokens=True))
    cands.extend(c)
    lab.extend([[labels[i]]]*10)
    eval_arg = [inpu,cands,lab]
    if i%100==0:
        metric = metrics_func(eval_arg)
        print(metric)


print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)


  0%|          | 0/10000 [00:00<?, ?it/s]/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


10 10 10
3


  0%|          | 1/10000 [00:04<13:09:15,  4.74s/it]

(40.63008858858876, 40.81037555988784)


  1%|          | 100/10000 [05:09<7:32:35,  2.74s/it]

...

13010 13010 13010
3


 13%|█▎        | 1301/10000 [1:12:16<23:26:18,  9.70s/it]

(51.05227808128896, 44.47899453969827)


 13%|█▎        | 1321/10000 [1:13:12<8:00:59,  3.33s/it] 


KeyboardInterrupt: 

In [ ]:
import pandas as pd
test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()


metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1

for  i  in tqdm(range(len(texts))):
    text = texts[i]
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=1,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
   # print("====================================")
    inpu.extend([text]*1)
    c =[]
    for x, beam in enumerate(beam_outputs):
        #print("{}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
        c.append(tokenizer.decode(beam, skip_special_tokens=True))
    cands.extend(c)
    lab.extend([[labels[i]]]*1)
    eval_arg = [inpu,cands,lab]
    if i%100==0:
        metric = metrics_func(eval_arg)
        print(metric)


print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)


In [7]:
text = "Since,2010, project researchers have uncovered documents in portugal that have revealed who owned the ship."
    # encode the text into tensor of integers using the appropriate tokenizer
inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
    no_repeat_ngram_size=3,
    num_return_sequences=10,
    top_k=4, top_p=0.95
    # return_dict_in_generate=True,
)
for x, beam in enumerate(beam_outputs):
    print("{} {}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))

1321 Since, 2010, project researchers have uncovered documents in portugal that have revealed who owned the ship.
1321 Since 2010, project researchers have uncovered documents in portugal that have revealed who owned the ship.
1321 Project researchers have uncovered documents in portugal that have revealed who owned the ship.
1321 Since, 2010, project researchers have uncovered documents in portugal that have revealed who owned the ship.
1321 Since, 2010, project researchers have uncovered documents in portugal which have revealed who owned the ship.
1321 Since 2010, project researchers have uncovered documents in portugal which have revealed who owned the ship.
1321 Since, 2010, project researchers have uncovered documents in portugal that have revealed who owns the ship.
1321 Since 2010, project researchers have uncovered documents in portugal that have revealed who owns the ship.
1321 Since,2010, project researchers have uncovered documents in portugal that have revealed who owned t

In [6]:
text = "Experts say China's air pollution exacts a tremendous toll on human health."
    # encode the text into tensor of integers using the appropriate tokenizer
inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
    no_repeat_ngram_size=3,
    num_return_sequences=10,
    top_k=4, top_p=0.95
    # return_dict_in_generate=True,
)
for x, beam in enumerate(beam_outputs):
    print("{} {}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))

1321 Some people say China's air pollution has a great toll on human health.
1321 Some people say China's air pollution has a great effect on human health.
1321 Some people say China's air pollution is a tremendous toll on human health.
1321 Some people say China's air pollution has a great impact on human health.
1321 Some people say China's air pollution exacts a tremendous toll on human health.
1321 Some people say that China's air pollution has a great toll on human health.
1321 Some people say that China's air pollution has a great effect on human health.
1321 Some people say China's air pollution is a great toll on human health.
1321 Some experts say China's air pollution has a great toll on human health.
1321 They say China's air pollution has a great toll on human health.


## controled